In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm.notebook import tqdm

# Overview

This notebook shows a basic prediction using last week's best sellers and includes calculation of MAP@12 and a simple four fold validation.

# Data Loading

Uses some preprocessed data with minor feature engineering applied. See the data set [H&M Fashion Parquet Performance](https://www.kaggle.com/datasets/tbierhance/hm-fashion-recommendation-parquet).

In [ ]:
customers = pd.read_parquet('../input/hm-fashion-recommendation-parquet/customers.parquet')
articles = pd.read_parquet('../input/hm-fashion-recommendation-parquet/articles.parquet')
sales = pd.read_parquet('../input/hm-fashion-recommendation-parquet/sales.parquet')
customer_ids = pd.read_parquet('../input/hm-fashion-recommendation-parquet/customer_ids.parquet')
sample_submission = pd.read_parquet('../input/hm-fashion-recommendation-parquet/sample_submission.parquet')

In [ ]:
sales.head()

# Metric: Mean Average Precision @ k




The metric MAP@k as defined for this competition can be calculated from the average precision @ k for every customer. Here is a simple implementation of the metric. It does NOT handle duplicates:

In [ ]:
def average_precision_score(y_true, y_score, k=None):
    if k is None: k=len(y_score)
    relevant = np.isin(y_score[:k], y_true) # relevant[i]==1 if y_score[i] is correct
    patk = np.cumsum(relevant)/np.arange(1, len(y_score[:k])+1) # patk[0]==P@1, patk[1]==P@2, ...
    return(np.sum(patk*relevant)/min(len(y_true), k)) # as defined by the competition

Some examples (using 4 predictions instead of 12 for the sake of clarity):

In [ ]:
# Example 1: all predictions are wrong
y_true = [1, 2, 3, 4]
y_score = [5, 6, 7, 8]
average_precision_score(y_true, y_score)

In [ ]:
# Example 2: all predictions are correct (prediction order does NOT matter)
y_true = [1, 2, 3, 4]
y_score = [4, 3, 2, 1]
average_precision_score(y_true, y_score)

In [ ]:
# Example 3: first prediction is incorrect (prediction order DOES matter, ground truth order DOES NOT matter)
y_true = [1, 2, 3, 4]
y_score = [0, 2, 3, 4]
average_precision_score(y_true, y_score)

In [ ]:
# Example 4: last prediction is incorrect (prediction order DOES matter, ground truth order DOES NOT matter)
y_true = [1, 2, 3, 4]
y_score = [1, 2, 3, 5]
average_precision_score(y_true, y_score)

In [ ]:
# Example 5: y_true can be shorter than k
y_true = [3, 4]
y_score = [1, 2, 3, 4]
average_precision_score(y_true, y_score)

In [ ]:
# Example 6: the ground truth y_true can be longer than k, however y_score should be truncated to the first k entries
y_true = [1, 2, 3, 4, 5, 6]
y_score = [1, 2, 6, 7, 5, 3]
print(f'AP@6: {average_precision_score(y_true, y_score, k=6):.4f}')
print(f'AP@4: {average_precision_score(y_true, y_score, k=4):.4f}') # predictions 5 and 3 are being ignored

# Train & Validate: use best sellers only

This methods derives the best selling articles for the week before the validation week. All customers that bought some article in the validation week will be scored. Returns the MAP@12 over all customers.

In [ ]:
def train_validate(validation_week):
    # get the best selling articles in the week before the prediction
    train = sales[sales.week == validation_week-1]
    best_sellers = train.groupby('article_id').size().nlargest(12).index.values
    
    # only include customers that bought something in the week to predict
    validate = sales[sales.week==validation_week].groupby('customer_id').article_id.unique().reset_index(name='y_true')
    # use best selling articles for the prediction for every customer
    validate['y_score'] = validate.apply(lambda x: best_sellers, axis=1)
    # calculate AP@12 for every customer
    validate['ap@12'] = validate.apply(lambda row: average_precision_score(row['y_true'], row['y_score']), axis=1)
    # return MAP@12 over all customers
    return(validate['ap@12'].mean())

Four fold validation using the week 104 just before the test week 105, weeks 52 and 53 that are similar to the test week (one year ago) and week 78 which is in between.

In [ ]:
validation_weeks = [52, 53, 78, 104]
results=[]
for (idx, validation_week) in enumerate(tqdm(validation_weeks)):
    result = train_validate(validation_week)
    print(f'Fold {idx} predicting week {validation_week}: MAP@12={result:.4f}')
    results.append(result)
print()
print(f'Over all folds: MAP@12={np.mean(results):.4f}')

# Final train

Derive best sellers for the last week.

In [ ]:
TEST_WEEK = 105
train = sales[sales.week == TEST_WEEK-1]
best_sellers = train.groupby('article_id').size().nlargest(12).index.values

Predict best sellers for every customer.

In [ ]:
submission = customers[['customer_id']].copy()
submission['y_score'] = submission.apply(lambda x: best_sellers, axis=1)

Remap the custom integer customer_ids to the original customer_ids and format the list of articles.

In [ ]:
submission = submission.merge(customer_ids)[['customer_id_original', 'y_score']].rename(columns={'customer_id_original': 'customer_id', 'y_score': 'prediction'})
submission['prediction'] = submission.prediction.apply(lambda x: ' '.join([f'{e:010d}' for e in x]))
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)